In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# parity check matrix H
H = np.array([[1, 1, 1, 1, 0, 0],
              [0, 0, 1, 1, 0, 1],
              [1, 0, 0, 1, 1, 0]])

In [3]:
H.shape

(3, 6)

https://en.wikipedia.org/wiki/Low-density_parity-check_code

In [4]:
def generate_H_hat(H):
    
    rows, cols = H.shape
    N, K = cols, cols - rows
    M = N - K
    H_hat = H.copy()
    
    for i in range(M):
        j = 1
        while H_hat[i][K + i] != 1:
            flag = (i + j) % M
            sum_ = H_hat[i] + H_hat[flag]
            H_hat[i] = sum_ % 2
            j += 1
            
        for k in range(M):
            if k == i:
                continue
            if H_hat[k][K + i] == 1:
                sum_ = H_hat[i] + H_hat[k]
                H_hat[k] = sum_ % 2
    return H_hat

In [5]:
def generate_G(H):
    
    rows, cols = H.shape
    N, K = cols, cols - rows
    M = N - K
    H_hat = generate_H_hat(H)
    
    I = np.eye(K, dtype=int)
    P = H_hat[:, :K]
    G = np.vstack((I, P))

    return G

In [6]:
H_hat = generate_H_hat(H)
G = generate_G(H)

print("H_hat:")
print(H_hat)
print("\nG:")
print(G)

H_hat:
[[1 1 1 1 0 0]
 [0 1 1 0 1 0]
 [1 1 0 0 0 1]]

G:
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 1]
 [0 1 1]
 [1 1 0]]


In [7]:
# read files
H1 = np.loadtxt(fname="H1.txt", dtype=int)
y1 = np.loadtxt(fname="y1.txt", dtype=int)

In [8]:
H1.shape

(750, 1000)

In [9]:
y1.shape

(1000,)

In [10]:
# read files
H1 = np.loadtxt(fname="H1.txt", dtype=int)
y1 = np.loadtxt(fname="y1.txt", dtype=int)

In [11]:
def decoder(H, y, p=0.1):

    rows, cols = H.shape
    N, K = cols, cols - rows
    M = N - K
    nums_y = len(y)
    
    p_x1 = np.zeros(shape=nums_y)
    p_x0 = np.zeros(shape=nums_y)
    loglik = np.zeros(shape=(rows, cols))
    bits = []
    itera_times = 0
    
    for i in range(nums_y):
        p_x1[i] = p**((y[i]+1)%2) * (1-p)**((y[i]+1+1)%2)
        p_x0[i] = p ** y[i] * (1 - p) ** ((y[i] + 1) % 2)
#     print(p_x1)
#     print(p_x0)
    
    # log likelihood will be faster
    for row in range(rows):
        bits.append(np.where(H[row]==1)[0])
        for col in range(cols):
            loglik[row, col] = np.log(p_x0[col] / p_x1[col])
    
#     print(loglik)

    while True:
        itera_times += 1
        
        loop_continued = False
        loglik_factorized = np.zeros(shape=(rows, cols))
        pred = np.zeros(shape=cols)
        
        for row in range(rows):
            column = np.zeros(shape=cols)
            prod = loglik[row, bits[row]]
            
            for i in range(len(prod)):
                prod_ = np.prod(np.tanh(np.hstack((prod[0:i], prod[i + 1:len(prod)])) / 2))
#                 prod_ = 1
#                 for j in temp:
#                     prod_ *= j
                column[bits[row][i]] = np.log((1 + prod_) / (1 - prod_))
    
            loglik_factorized[row, :] = column
        
#         print(loglik_factorized)
        posterior = loglik_factorized.sum(axis=0)
        for i in range(len(posterior)):
            posterior[i] += np.log(p_x0[i] / p_x1[i])
        print(posterior)   
        for i in range(len(pred)):
            pred[i] = 0 if posterior[i] > 0 else 1
        
        
        temp = np.matmul(H, pred.T) % 2
        for element in temp:
            if element != 0:
                loop_continued = True
                break
        if loop_continued:
            for row in range(rows):
                for col in range(cols):
                    loglik[row, col] = posterior[col] - loglik_factorized[row, col]
        else:
            break
            
    return pred


In [12]:
decoded_vector = decoder(H1, y1)
print(decoded_vector)

[ 3.32809773 -5.58984403  3.32809773  3.32809773 -1.19539488  1.19539488
  3.32809773  1.06635143  5.58984403 -3.32809773 -1.06635143  3.32809773
  3.32809773  3.32809773  3.32809773 -5.58984403  5.58984403 -5.58984403
 -5.58984403 -3.32809773  1.06635143  1.06635143  3.32809773  3.32809773
 -1.19539488 -3.32809773 -5.58984403 -1.06635143  1.19539488  3.32809773
  3.32809773  5.58984403  3.32809773 -5.58984403 -5.58984403 -5.58984403
 -5.58984403  3.32809773  3.32809773 -5.58984403  5.58984403  1.06635143
 -3.32809773 -1.06635143  3.32809773  5.58984403  5.58984403  5.58984403
 -1.06635143 -3.32809773  1.06635143  3.32809773 -3.32809773 -3.32809773
 -1.19539488 -1.19539488  3.32809773 -5.58984403 -5.58984403  5.58984403
 -5.58984403 -1.06635143 -3.32809773 -1.19539488  5.58984403 -5.58984403
 -3.32809773  5.58984403 -5.58984403  1.19539488  1.06635143  3.32809773
  3.32809773 -3.32809773 -5.58984403  5.58984403 -5.58984403  1.06635143
 -1.19539488  1.06635143  5.58984403 -5.58984403 -5

[  7.02884179  -3.55249641   8.96717375   7.71988027  -4.50848599
   4.44768583  12.86891271   4.86990692  10.74009806  -2.22066958
  -3.72072194   4.14530723   5.46477674   4.63391955   6.6474865
 -10.10302439   9.30560977  -8.67700608  -7.19304672 -10.02327806
   4.00248907   0.32145135   2.86266896   4.63150786   0.22806239
  -6.01251483  -6.8005419   -7.03857794   4.97667242   5.39511387
   6.2201464    7.305125     8.48202144 -12.58747274  -7.86612793
 -10.28614344 -10.47910325   8.14222434   8.90696917 -11.64691951
   4.55752014   1.63861727  -7.23215857   1.33229052   2.49112759
   9.09230424   8.13584059  10.65268811   2.73547114  -2.10223721
   3.73124782   2.56781518  -7.88961101   0.21555127  -6.05210857
   3.49404121   3.8917662  -12.20726618  -7.51862125   9.78067136
  -8.07625319   0.03358589  -5.08049802  -2.1242529   13.29958575
  -6.56911668  -5.31209074   7.28400706  -7.42205764   1.09045745
  -0.10364143   5.65598699   6.61345774  -6.62211599  -9.44013908
   8.691543

[ 1.03132723e+01 -2.34792779e+00  8.43138039e+00  4.61524274e+00
 -9.17885955e+00  7.61997615e+00  1.30281799e+01  6.59569099e+00
  1.37107202e+01 -9.55604965e+00 -1.14787730e+01  1.10594505e+01
  7.15671166e+00  6.13981775e+00  7.45182466e+00 -1.43121802e+01
  1.15117758e+01 -7.96926269e+00 -5.90122621e+00 -1.18790983e+01
  6.51134063e+00  3.01413128e+00  6.74501080e+00  8.91640996e+00
  7.74982671e+00 -1.40289873e+01 -8.14842883e+00 -1.20204320e+01
  5.40772858e+00  5.88392121e+00  6.75979898e+00  8.73682138e+00
  1.43395404e+01 -1.13905457e+01 -1.16181715e+00 -8.05797616e+00
 -8.90695842e+00  1.09126479e+01  1.29220762e+01 -1.28652459e+01
  4.32743444e+00 -2.65066257e+00 -6.65584949e+00  5.93317008e+00
  1.44846881e+00  1.02794020e+01  1.74155816e+01  1.55886579e+01
  1.11044526e+01 -1.99321718e+00  6.59450238e+00  5.15341063e+00
 -1.88367986e+01  1.93510235e+00 -2.35734892e+00 -2.23621916e+00
  1.34598415e+01 -1.52175651e+01 -1.26288758e+01  1.68767096e+01
 -1.43601121e+01 -2.48217

[ 16.41480455  -3.16098015  11.58288517   8.77857781 -14.82090385
  17.31261483  22.76912316  17.49686145  21.13900143 -16.60131048
 -21.85483613  13.44868185  19.41309891  11.90948468   6.29409665
 -19.62713137  17.51274232 -22.15818805 -20.5294927  -23.51593343
  14.6070272   11.29310166  16.55011562  11.20749894  17.29316557
 -22.75259204 -20.79627452 -14.49180988  19.56864032  23.30339614
  28.26436594  18.79633725  17.92224663 -18.96515341 -15.07180293
 -18.92250692 -14.38419591  21.15976276  21.61226551 -26.93200129
  21.24790631  10.52202298 -19.83686735  13.05724799  18.50611377
  23.27048387  22.79905173  19.6336329    5.76224957 -16.00558749
  11.06139174   9.25157894 -15.33594737   8.61501652  10.14775614
   6.11247656   6.50280974 -22.63235385 -22.55773797  28.18459473
 -21.33061026 -12.33424342 -18.25090145 -11.82180209  22.49407441
 -10.82738284  -7.83622112  10.07978282  -9.74944066  -5.37611767
  10.13929162  17.66798521  17.88122969 -18.04469583 -18.63189969
  18.84891